In [2]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv("../total_dataset_final.csv")

In [1]:
!pip install lightgbm scikit-learn pandas numpy matplotlib --quiet


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
df

,timestamp,SO2,CO,O3,NO2,PM10,PM25,WS,PS,TA,...,RN_t_minus_1,RN_t_minus_2,VS_t_minus_1,VS_t_minus_2,Traffic_t_minus_1,Traffic_t_minus_2,WD_sin_t_minus_1,WD_sin_t_minus_2,WD_cos_t_minus_1,WD_cos_t_minus_2
0,2023-01-01 02:00:00,0.0040,1.00,0.0020,0.0470,52.0,52.0,1.9,1028.7,1.5,...,0.0,0.0,1104.0,1503.0,3950.0,3873.0,-0.939693,-0.342020,-3.420201e-01,-9.396926e-01
1,2023-01-01 03:00:00,0.0040,1.30,0.0020,0.0500,57.0,54.0,1.6,1029.0,1.6,...,0.0,0.0,925.0,1104.0,2389.0,3950.0,-0.939693,-0.939693,-3.420201e-01,-3.420201e-01
2,2023-01-01 04:00:00,0.0040,1.30,0.0020,0.0520,67.0,57.0,1.4,1029.0,1.5,...,0.0,0.0,794.0,925.0,1916.0,2389.0,-0.939693,-0.939693,-3.420201e-01,-3.420201e-01
3,2023-01-01 05:00:00,0.0040,0.60,0.0130,0.0230,71.0,61.0,2.4,1029.0,0.8,...,0.0,0.0,709.0,794.0,1679.0,1916.0,-0.939693,-0.939693,3.420201e-01,-3.420201e-01
4,2023-01-01 06:00:00,0.0040,0.60,0.0140,0.0260,51.0,41.0,3.1,1029.3,-0.2,...,0.0,0.0,1185.0,709.0,2070.0,1679.0,-0.939693,-0.939693,3.420201e-01,3.420201e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17535,2024-12-31 17:00:00,0.0032,0.53,0.0217,0.0239,27.0,11.0,3.4,1020.9,2.5,...,0.0,0.0,3396.0,4227.0,6706.0,7294.0,-0.939693,-1.000000,-3.420201e-01,-1.836970e-16
17536,2024-12-31 18:00:00,0.0032,0.57,0.0187,0.0268,29.0,12.0,3.1,1021.6,1.2,...,0.0,0.0,3570.0,3396.0,5978.0,6706.0,-1.000000,-0.939693,-1.836970e-16,-3.420201e-01
17537,2024-12-31 19:00:00,0.0032,0.69,0.0162,0.0329,31.0,14.0,1.9,1022.1,0.5,...,0.0,0.0,3672.0,3570.0,5907.0,5978.0,-0.939693,-1.000000,-3.420201e-01,-1.836970e-16
17538,2024-12-31 20:00:00,0.0030,0.50,0.0197,0.0254,27.0,15.0,2.4,1022.3,0.0,...,0.0,0.0,3760.0,3672.0,6231.0,5907.0,-1.000000,-0.939693,-1.836970e-16,-3.420201e-01


In [8]:
df["timestamp"] = pd.to_datetime(df["timestamp"])

# 타겟과 피처 확인
targets = ["PM25_t_plus_1","PM25_t_plus_2","PM25_t_plus_3"]
assert all(t in df.columns for t in targets), "타겟 컬럼 누락 확인 필요"

# 간단 점검
print(df.shape, df.isna().mean().sort_values(ascending=False).head(10))
df.head(3)

(17540, 49) timestamp    0.0
SO2          0.0
CO           0.0
O3           0.0
NO2          0.0
PM10         0.0
PM25         0.0
WS           0.0
PS           0.0
TA           0.0
dtype: float64


,timestamp,SO2,CO,O3,NO2,PM10,PM25,WS,PS,TA,...,RN_t_minus_1,RN_t_minus_2,VS_t_minus_1,VS_t_minus_2,Traffic_t_minus_1,Traffic_t_minus_2,WD_sin_t_minus_1,WD_sin_t_minus_2,WD_cos_t_minus_1,WD_cos_t_minus_2
0,2023-01-01 02:00:00,0.004,1.0,0.002,0.047,52.0,52.0,1.9,1028.7,1.5,...,0.0,0.0,1104.0,1503.0,3950.0,3873.0,-0.939693,-0.342020,-0.34202,-0.939693
1,2023-01-01 03:00:00,0.004,1.3,0.002,0.050,57.0,54.0,1.6,1029.0,1.6,...,0.0,0.0,925.0,1104.0,2389.0,3950.0,-0.939693,-0.939693,-0.34202,-0.342020
2,2023-01-01 04:00:00,0.004,1.3,0.002,0.052,67.0,57.0,1.4,1029.0,1.5,...,0.0,0.0,794.0,925.0,1916.0,2389.0,-0.939693,-0.939693,-0.34202,-0.342020


In [9]:
n = len(df)
train_end, val_end = int(n*0.7), int(n*0.85)
train_df = df.iloc[:train_end].copy()
val_df   = df.iloc[train_end:val_end].copy()
test_df  = df.iloc[val_end:].copy()

feature_cols = [c for c in df.columns if c not in targets + ["timestamp"]]

In [ ]:
import numpy as np

def persistence_baseline(y_true, horizon):  
    y_pred = y_true.shift(horizon)  

    mask = ~y_pred.isna()
    return y_true[mask].values, y_pred[mask].values

In [11]:
import lightgbm as lgb
import math
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def rmse(y_true, y_pred):
    return math.sqrt(mean_squared_error(y_true, y_pred))

def mape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask]-y_pred[mask]) / y_true[mask]))*100

X_train, X_val, X_test = train_df[feature_cols], val_df[feature_cols], test_df[feature_cols]

params = dict(
    n_estimators=3000,
    learning_rate=0.03,
    num_leaves=63,
    min_data_in_leaf=50,
    subsample=0.9,            # = bagging_fraction
    colsample_bytree=0.9,     # = feature_fraction
    random_state=42,
    n_jobs=-1
)

In [13]:
results = []

for tcol in targets:
    y_train, y_val, y_test = train_df[tcol], val_df[tcol], test_df[tcol]

    model = lgb.LGBMRegressor(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric="l2",                 # RMSE 기반과 동일 계열 (l2 = MSE)
        callbacks=[lgb.early_stopping(200), lgb.log_evaluation(0)]  # ✅ verbose 제거 후 콜백으로 대체
    )

    y_pred = model.predict(X_test, num_iteration=model.best_iteration_)
    mae = mean_absolute_error(y_test, y_pred)
    rmse_ = rmse(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mape_ = mape(y_test, y_pred)

    results.append([tcol, mae, rmse_, r2, mape_])

[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[367]	valid_0's l2: 11.7833
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[275]	valid_0's l2: 19.3326
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[196]	valid_0's l2: 26.9737


In [ ]:
print(f"{tcol:15s} | MAE={mae:.3f} | RMSE={rmse_:.3f} | R²={r2:.3f} | MAPE={mape_:.2f}%")

PM25_t_plus_3   | MAE=3.344 | RMSE=5.013 | R²=0.714 | MAPE=33.99%


| 지표            | 값      | 의미           | 해석                                                                                |
| ------------- | ------ | ------------ | --------------------------------------------------------------------------------- |
| **MAE**       | 3.300  | 평균 절대 오차     | 예측값이 실제값에서 평균적으로 약 **±3.3 µg/m³** 정도 오차가 났다는 뜻                                    |
| **RMSE**      | 4.973  | 제곱 평균 제곱근 오차 | 큰 오차에 더 가중을 주는 지표. 예측이 실제보다 **약 ±5 µg/m³** 정도 흔들림이 있었다는 뜻                         |
| **R² (결정계수)** | 0.718  | 설명력          | 모델이 실제 PM2.5 변동의 **약 71.8%를 설명**하고 있음. 나머지 약 28.2%는 모델이 잡지 못한 요인                  |
| **MAPE (%)**  | 33.24% | 평균 상대 오차     | 예측값이 실제값에서 평균적으로 **약 33% 정도 비율의 오차**를 냈다는 뜻. 즉 예를 들어 실제가 15면 예측이 ±5 정도 차이 났다는 의미. |
